1. Create a demo copy of the survey in SurveyMonkey for development and testing. 

2. Test the SurveyMonkey API.

3. Create Flask Web App.

---

#### 1. **Create a demo survey**

We created a demo survey in SurveyMonkey by copy-pasting questions and answers into the survey creation box: 


In [ ]:
import os
import json
import pandas as pd 

# Create matching survey in SM: import through text import (limited to 10 questions for free account)
with open(os.path.join("data","get-skills-api","response.json"), "r") as file: # previously extracted with API Explorer
    survey_questions = json.load(file)['Skills']

skill_level_map = { 
  "DataPoint20": "Beginner",
  "DataPoint35": "Basic",
  "DataPoint50": "Skilled",
  "DataPoint65": "Advanced",
  "DataPoint80": "Expert"
}

for q in survey_questions:
  for k,v in skill_level_map.items():
    q[v] = q.pop(k)

# Test format for copy pasting into SM web console
with open("survey-question-port.txt", "w") as file:
  for q in survey_questions[:9]:
    file.write(f"{q['ElementName']}: {q['Question']}\n")
    file.write(f"1 - Beginner: {q['AnchorFirst']}\n")
    file.write("2\n")
    file.write(f"3 - Skilled: {q['AnchorThrid']}\n")
    file.write("4\n")
    file.write(f"5 - Expert: {q['AnchorLast']}\n")
    file.write("\n")

![create-survey](img/create-survey.png)

#### 2. **Testing Survey Monkey API**

1. [] Register draft app (public for test, private requires enterprise license)
2. [] create website to host app
3. [] set up [oauth redirect](https://developer.surveymonkey.com/api/v3/#authentication) to app from host site 

**Step 1:** Send the user whose SurveyMonkey account you wish to access to a specially crafted OAuth link. The page presented to the user will identify your app, ask them to log into SurveyMonkey if they aren’t already, and ask them to authorize any required scopes.

The OAuth link should be https://api.surveymonkey.com/oauth/authorize with urlencoded parameters: redirect_uri, client_id, response_type and state.

* "response_type"="code" (always, hardcoded value)
* "client_id"=unique client id you got when registering your app
* "redirect_uri"=URL encoded OAuth redirect URI you registered for your app
* "state" (recommended)=A value included in the request that will also be returned in the token response. It can be a string of any content that you wish. A randomly generated unique value is typically used for preventing cross-site request forgery attacks.*

e.g. `https://api.surveymonkey.com/oauth/authorize?response_type=code&redirect_uri=https%3A%2F%2Fapi.surveymonkey.com%2Fapi_console%2Foauth2callback&client_id=SurveyMonkeyApiConsole%2Fstate=uniquestring`

In [9]:
import yaml 
from urllib.parse import urlencode

oauth_link = "https://api.surveymonkey.com/oauth/authorize"

with open("api-key.yaml","r") as file:
    api_info = yaml.full_load(file)

RESPONSE_TYPE = "code"
REDIRECT_URI = "http://localhost:5000/auth/callback" # for local test
CLIENT_ID = api_info['sm']['app']['client-id']
# SECRET = api_info['sm']['app']['secret']
# ACCESS_TOKEN = api_info['sm']['app']['access_token']
print(RESPONSE_TYPE, REDIRECT_URI, CLIENT_ID)
params = {
    'response_type': RESPONSE_TYPE, 
    'redirect_uri': REDIRECT_URI, 
    'client_id': CLIENT_ID
}

full_oauth_link = f"{oauth_link}?{urlencode(params)}"
full_oauth_link

code http://127.0.0.1:5000/auth/callback C_44TfLERSi0kXy-tS2snQ


'https://api.surveymonkey.com/oauth/authorize?response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A5000%2Fauth%2Fcallback&client_id=C_44TfLERSi0kXy-tS2snQ'